## Group Member
### Abha Chaudhary
### Amit Luhar
### Hozefa Lakdawala

### The usual, Imports

In [2]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image
from torchsummary import summary
import torch.nn.functional as F
torch.cuda.empty_cache()

### Displaying the graph

In [2]:
def plot_the_graph(train_losses,val_losses,train_accuracies,val_accuracies,num_epochs):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, num_epochs+1), train_losses, label='Train')
    plt.plot(range(1, num_epochs+1), val_losses, label='Validation')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(range(1, num_epochs+1), train_accuracies, label='Train Accuracy', color='magenta')
    plt.plot(range(1, num_epochs+1), val_accuracies, label='Validation Accuracy', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Validation Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

### Training Loop

In [3]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        correct_train = 0
        total_train = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data,1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        train_loss = train_loss / len(train_loader.dataset)
        train_acc = correct_train / total_train
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = correct / total
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
    return train_losses,val_losses,train_accuracies,val_accuracies

### Testing Loop

In [4]:
def test_model(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
    print(f"Test Accuracy: {test_accuracy:.4f}")

### Loading and PreProcessing the Data

In [7]:
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((48, 48)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])

In [8]:
train_path = "FER-13/train"
test_path = "FER-13/test"

train_dataset = torchvision.datasets.ImageFolder(root=train_path, transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root=test_path, transform=transform)

train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [7]:
batch_size = 6144
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, pin_memory=True, shuffle=True)
# train_gpu_loader = [(inputs.to(device), labels.to(device)) for inputs, labels in train_loader]
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, pin_memory=True, shuffle=False)
# val_gpu_loader = [(inputs.to(device), labels.to(device)) for inputs, labels in val_loader]
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
# test_gpu_loader = [(inputs.to(device), labels.to(device)) for inputs, labels in test_loader]
print(device)

cuda:0


In [8]:
count = 0
for input,lab in train_loader:
    count+=1
    if(count > 4):
        break

# count = 0
# for input,lab in train_gpu_loader:
#     print(input.device)
#     count+=1
#     if(count > 4):
#         break    

### Show Loaded Data

In [9]:
classes = os.listdir(train_path)
fig, axs = plt.subplots(1, 7, figsize=(15, 5))
for i, class_name in enumerate(classes):
    img_path = os.path.join(train_path, class_name, os.listdir(os.path.join(train_path, class_name))[0])
    img = Image.open(img_path)
    axs[i].imshow(img, cmap='gray')
    axs[i].set_title(class_name)
    axs[i].axis('off')
plt.show()

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


### Creating the SimpleCNN Model

In [10]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=7,dropout=False):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=5, padding=2)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 3 * 3, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, num_classes)
        if dropout:
            self.dropout = torch.nn.Dropout(0.5)
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = self.pool(torch.relu(self.conv4(x)))
        x = torch.flatten(x,1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = SimpleCNN(dropout=False).to(device)
print(model)
summary(model,(1,48,48))

SimpleCNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv4): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2304, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=7, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 48, 48]             832
         MaxPool2d-2           [-1, 32, 24, 24]               0
            Conv2d-3           [-1, 64, 24, 24]          51,264
         MaxPool2d-4           [-1, 64, 12, 12]               0
            Conv2d-5

### Training Simple CNN model

In [11]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
n_epochs = 50
train_losses,val_losses,train_accuracies,val_accuracies = train_model(model, train_loader, val_loader, criterion,optimizer, num_epochs=n_epochs)

Epoch 1/50, Train Loss: 2.0445, Val Loss: 1.9225, Train Acc: 0.2261, Val Acc: 0.1421
Epoch 2/50, Train Loss: 1.9265, Val Loss: 1.9251, Train Acc: 0.1428, Val Acc: 0.1421
Epoch 3/50, Train Loss: 1.9207, Val Loss: 1.9018, Train Acc: 0.1637, Val Acc: 0.1843
Epoch 4/50, Train Loss: 1.8895, Val Loss: 1.8538, Train Acc: 0.1717, Val Acc: 0.1843
Epoch 5/50, Train Loss: 1.8388, Val Loss: 1.8121, Train Acc: 0.1762, Val Acc: 0.2459
Epoch 6/50, Train Loss: 1.8133, Val Loss: 1.8211, Train Acc: 0.2519, Val Acc: 0.2459
Epoch 7/50, Train Loss: 1.8161, Val Loss: 1.8163, Train Acc: 0.2519, Val Acc: 0.2459
Epoch 8/50, Train Loss: 1.8101, Val Loss: 1.8120, Train Acc: 0.2519, Val Acc: 0.2459
Epoch 9/50, Train Loss: 1.8097, Val Loss: 1.8116, Train Acc: 0.2519, Val Acc: 0.2459
Epoch 10/50, Train Loss: 1.8081, Val Loss: 1.8087, Train Acc: 0.2519, Val Acc: 0.2459
Epoch 11/50, Train Loss: 1.8066, Val Loss: 1.8222, Train Acc: 0.2519, Val Acc: 0.2459
Epoch 12/50, Train Loss: 1.8146, Val Loss: 1.8115, Train Acc: 0

KeyboardInterrupt: 

### Displaying the Graph

In [ ]:
plot_the_graph(train_losses=train_losses,val_losses=val_losses,train_accuracies=train_accuracies,val_accuracies=val_accuracies,num_epochs=n_epochs)

### Evaluating the simple cnn model on Test dataset

In [ ]:
test_model(model, test_loader)

### Saving the Model

In [ ]:
model_path = "simple_cnn_model.pth"
torch.save(model.state_dict(),model_path)

### Training the Model with droupouts and adam optimizer

In [ ]:
model = SimpleCNN().to(device)
print(model)
criterion = torch.nn.CrossEntropyLoss()
optimizer_adam = optim.Adam(model.parameters(), lr=0.001,weight_decay=0.0005)
n_epochs = 50
train_losses,val_losses,train_accuracies,val_accuracies = train_model(model, train_loader, val_loader, criterion, optimizer_adam, num_epochs=n_epochs)

### Evaluating the cnn model with dropout on Test dataset

In [ ]:
test_model(model, test_loader)

### Displaying the Graph with adam

In [ ]:
plot_the_graph(train_losses=train_losses,val_losses=val_losses,train_accuracies=train_accuracies,val_accuracies=val_accuracies,num_epochs=n_epochs)

### Saving the Model with dropouts

In [ ]:
model_path = "cnn_model_with_dropout.pth"
torch.save(model.state_dict(),model_path)

### Training the Model with droupouts and sgd optimizer

In [ ]:
model = SimpleCNN(dropout=True).to(device)
print(model)
criterion = torch.nn.CrossEntropyLoss()
optimizer_sgd = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [ ]:
n_epochs = 50
train_losses,val_losses,train_accuracies,val_accuracies = train_model(model, train_loader, val_loader, criterion,optimizer_sgd, num_epochs=n_epochs)

### Displaying the Graph with SGD

In [ ]:
plot_the_graph(train_losses=train_losses,val_losses=val_losses,train_accuracies=train_accuracies,val_accuracies=val_accuracies,num_epochs=n_epochs)

### Pre-trained Models

In [ ]:
AlexNet_Model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)
AlexNet_Model.eval()